# Using Kinematics from Reachy's URDF file

In [ ]:
import numpy as np

from reachy import Reachy

r = Reachy()

## Getting the Kinematic Chain from Reachy

We use the [Ikpy](https://github.com/Phylliade/ikpy) library to create the kinematic chain from the URDF. It uses:

* the path to the URDF file
* the base element of your chain
* the translation vector for the end tip

You can check the kinematic chain:

In [ ]:
for link in r.ik_chain._chain.links:
    print(link)

## Forward kinematic

You can then call the forward kinematic method by specifying all joint angle (in radians). You have to specify 9 joints in the case of Reachy: the 7 defined joints plus one virtual before and one virtual at the end.

For instance, when setting all joints to 0 except the elbow pitch at $\pi /2$:

In [ ]:
import numpy as np
np.set_printoptions(precision=3)

target = r.ik_chain.forward_kinematics([0, 0, 0, -90, 0, 0, 0])
print(target)

You can then extract the rotation matrix:

In [ ]:
rot = target[:3,:3]
print(rot)

And the translation vector:

In [ ]:
trans = target[:3, -1]
print(trans)

## Inverse kinematics

You can also use the inverse kinematics to find the joint angles from a target. The target must be passed as a 4x4 homogeneous matrix.

In [ ]:
rot = np.eye(3)
trans = [0.28, -0.09, -0.30]

target = np.eye(4)
target[:3, :3] = rot
target[:3, 3] = trans
print(target)

In [ ]:
joints = r.ik_chain.inverse_kinematics(target, accurate=True)
print(joints)

In [ ]:
def goto(joints):
    for i in range(len(joints)):
        r.motors[i].goal_position = joints[i]

In [ ]:
goto(joints)

You can also specify an initial position for the inverse kinematics optimisation. This will drastically speed up the process and help find better solutions.

In [ ]:
joints = r.ik_chain.inverse_kinematics(target, initial_position=[0, 0, 0, -90, 0, 0, 0], accurate=True)
print(joints)

In [ ]:
goto(joints)

In [ ]:
r.ik_chain.end_effector

## Interactive testing

Safety first!

In [ ]:
for m in r.motors:
    m.moving_speed = 50
    m.torque_limit = 75

In [ ]:
for m in r.motors:
    m.compliant = False

In [ ]:
def goto(x, y, z):
    rot = np.eye(3)
    trans = [x, y, z]

    target = np.eye(4)
    target[:3, :3] = rot
    target[:3, 3] = trans
    
    cur_pos = [m.present_position for m in r.motors]
    
    joints = r.ik_chain.inverse_kinematics(target, 
                                           initial_position=cur_pos)

    for i in range(len(r.motors)):
        r.motors[i].goal_position = joints[i]

In [ ]:
r.ik_chain.end_effector

In [ ]:
goto(...)

## Error measurement

In [ ]:
print(r.ik_chain.forward_kinematics([m.present_position for m in r.motors])[:3, 3])
print(r.ik_chain.forward_kinematics([m.goal_position for m in r.motors])[:3, 3])